# Modelling v2

modelling use the latest exploration (remove svm)

# Goal

Get the model from the prepared data. Without oversampling. 

# Plan

Checklist what will be done on the notebook :

    [*] Get Data
    [*] Modelling



In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
import time

from scipy.stats import uniform,randint
import warnings
warnings.filterwarnings("ignore")

In [2]:
######### run manually #############

# with open("../../config.yaml", "r") as f:
#     config = yaml.load(f, Loader=yaml.FullLoader)

# PATH_RAW = config['path']['raw']
# PATH_INTERIM = config['path']['interim']
# PATH_PROCESSED = config['path']['processed']

# if os.name == 'posix' :
#     PATH_MODEL = config['path']['artifact']+os.getcwd().split('/')[-1]+'/'
# else:
#     PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'

# PATH_UDF = config['path']['udf']

######### run with makefile #############

with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

PATH_RAW = config['path-make']['raw']
PATH_INTERIM = config['path-make']['interim']
PATH_PROCESSED = config['path-make']['processed']

if os.name == 'posix' :
    PATH_MODEL = config['path-make']['artifact']+'iteration_2/'
else:
    PATH_MODEL = config['path-make']['artifact']+'iteration_2/'

PATH_UDF = config['path-make']['udf']

In [3]:
from sklearn.linear_model import LogisticRegression, ElasticNet, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessRegressor

## Get Data

In [4]:
raw_data = pd.read_csv(PATH_PROCESSED+'prep_iter2.csv')
col_cat_load = joblib.load(PATH_MODEL+'cat_names.pkl')
col_num_load = joblib.load(PATH_MODEL+'num_names.pkl')

In [5]:
X_tr = raw_data[raw_data.is_train == 1][col_num_load+col_cat_load]
Y_tr = raw_data[raw_data.is_train == 1]['target'].values

X_val = raw_data[raw_data.is_train == 0][col_num_load+col_cat_load]
Y_val = raw_data[raw_data.is_train == 0]['target'].values

In [6]:
X_tr.shape

(32665, 26)

In [7]:
X_val.shape

(5765, 26)

In [8]:
Y_val.shape

(5765,)

## Modelling

In [9]:
proportion_cls = pd.Series(Y_tr).value_counts(normalize = True)
scale_pos_w = np.ceil(proportion_cls[0]/proportion_cls[1])
cw = {0:1, 1:scale_pos_w}
cw
# scale_pos_weight for xgboost

{0: 1, 1: 8.0}

In [10]:
proportion_cls

0    0.884525
1    0.115475
dtype: float64

In [11]:
all_model = {'sgd2' : SGDClassifier(class_weight=cw,max_iter = 2500,early_stopping=True,warm_start=True,eta0=0.1)
            ,'logreg2' : LogisticRegression(class_weight=cw)
            ,'rf2': RandomForestClassifier(class_weight=cw, random_state=234)
            ,'nb2':GaussianNB(priors=[0.884,0.116])
            }

In [12]:
param_options = {'logreg2' : dict(C=uniform(loc=0, scale=2)
                                 ,penalty=['l2', 'l1']
                                 ,solver=['liblinear', 'sag'])
                 ,'rf2' : dict(n_estimators=randint(300, 500)
                             ,max_depth=randint(20, 50)
                             ,min_samples_split=randint(5, 20)
                             ,min_samples_leaf=randint(5, 20)
                             ,min_weight_fraction_leaf=uniform(loc=0, scale=0.4)
                             ,min_impurity_decrease=uniform(loc=0, scale=0.3))
                 ,'nb2' : dict(var_smoothing = uniform(loc=1e-8, scale=1e-2))
                 # ,'gp2' : dict(alpha = uniform(loc=1e-12, scale=1e-5))
                 ,'sgd2' : dict(loss=['hinge', 'squared_hinge', 'perceptron']
                                ,penalty=['l2', 'l1']
                                ,alpha=uniform(loc=0.1, scale=0.95)
                                ,l1_ratio=uniform(loc=0.1, scale=0.75)
                                ,learning_rate=['constant','optimal','invscaling','adaptive']
                               )
                 # ,'el2': dict(alpha=uniform(loc=0.1, scale=2), l1_ratio=uniform(loc=0.1, scale=2))
                }

 # min_weight_fraction_leaf must in [0, 0.5]

In [13]:
## scoring
## https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
best_clf = {}
best_scr = {}
sla_model = {}
for mdl in all_model.keys():
    L1 = time.time()
    clf = all_model[mdl]
    parm = param_options[mdl]
    RS = RandomizedSearchCV(clf, parm, n_iter = 350, cv = 5, scoring = 'roc_auc',n_jobs = 10)
        
    RS.fit(X_tr,Y_tr)
    
    best_clf[mdl] = RS.best_estimator_
    best_scr[mdl] = RS.best_score_
    diff_time = round((time.time()-L1)/60, 2)
    
    joblib.dump(best_clf[mdl],PATH_MODEL+mdl+'.pkl')
    
    print(mdl, 'is done in ',diff_time, ' minutes')
    sla_model[mdl] = diff_time

sgd2 is done in  1.22  minutes


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coe

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coe

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

...

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for t

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for t

logreg2 is done in  17.07  minutes


rf2 is done in  10.79  minutes


nb2 is done in  0.34  minutes


In [14]:
sla_model

{'sgd2': 1.22, 'logreg2': 17.07, 'rf2': 10.79, 'nb2': 0.34}

In [15]:
best_scr

{'sgd2': 0.8461947157287332,
 'logreg2': 0.8549397152867447,
 'rf2': 0.8436456103942234,
 'nb2': 0.7911367688286551}

In [16]:
best_clf

{'sgd2': SGDClassifier(alpha=0.11566304626731856, class_weight={0: 1, 1: 8.0},
               early_stopping=True, eta0=0.1, l1_ratio=0.643704662358155,
               learning_rate='invscaling', loss='squared_hinge', max_iter=2500,
               warm_start=True),
 'logreg2': LogisticRegression(C=0.022940557277596252, class_weight={0: 1, 1: 8.0},
                    penalty='l1', solver='liblinear'),
 'rf2': RandomForestClassifier(class_weight={0: 1, 1: 8.0}, max_depth=22,
                        min_impurity_decrease=0.004518669879318071,
                        min_samples_leaf=10, min_samples_split=7,
                        min_weight_fraction_leaf=0.03140025220500258,
                        n_estimators=362, random_state=234),
 'nb2': GaussianNB(priors=[0.884, 0.116], var_smoothing=0.009979510749500424)}

## Evaluation

The SVM is best so far, but there is no predict proba.

In [17]:
for ml in best_clf.keys():
    Y_prob_pred = best_clf[ml].predict(X_val)
    print(ml)
    print(classification_report(Y_val,Y_prob_pred))

sgd2
              precision    recall  f1-score   support

           0       0.97      0.78      0.86      5079
           1       0.33      0.79      0.46       686

    accuracy                           0.78      5765
   macro avg       0.65      0.79      0.66      5765
weighted avg       0.89      0.78      0.81      5765

logreg2
              precision    recall  f1-score   support

           0       0.96      0.80      0.88      5079
           1       0.35      0.78      0.48       686

    accuracy                           0.80      5765
   macro avg       0.66      0.79      0.68      5765
weighted avg       0.89      0.80      0.83      5765

rf2
              precision    recall  f1-score   support

           0       0.97      0.69      0.81      5079
           1       0.27      0.84      0.41       686

    accuracy                           0.71      5765
   macro avg       0.62      0.77      0.61      5765
weighted avg       0.89      0.71      0.76      5765

nb

In [18]:
1-Y_val.mean()

0.8810060711188205